# 🤖 MCP Chat Application Workshop
## Building AI Assistants with Mistral models on Amazon Bedrock & Model Context Protocol

Welcome to this hands-on workshop! By the end of this session, you'll have built a fully functional AI chat application that connects Amazon Bedrock models with external tools via the Model Context Protocol (MCP).

### What You'll Learn:
- How to connect AI models to external tools and services
- Working with Mistral models on Amazon Bedrock
- Understanding the Model Context Protocol (MCP)
- Building chat interfaces using Gradio

### Prerequisites:
- Basic Python knowledge
- AWS account with Bedrock access
- Python 3.10+ installed

### Workshop Overview:
1. **Setup & Configuration** - Get your environment ready
2. **Understanding MCP** - Learn about Model Context Protocol
3. **Building the Core Chat** - Create a command-line chat interface
4. **Adding a Web Interface** - Build a Gradio-based web app
5. **Testing & Experimentation** - Try out your application

## Step 1: Setup & Environment Configuration

Let's start by setting up our environment and understanding what tools we'll need.

In [1]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 186.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 178.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 193.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 192.2 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-api90m━━━━━━━━━━━━━━━━━━━━━━ 12/27 [pydeck]olt]
    Found existing installation: opentelemetry-api 1.32.1━━━━━ 12/27 [pydeck]
    Uninstalling opentelemetry-api-1.32.1:━━━━━━━━━━━━━━━━━━━━ 12/27 [pydeck]
      Successfully uninstalled opentelemetry-api-1.32.1━━━━━━━ 12/27 [pydeck]
  Attempting uninstall: opentelemetry-semantic-conventions━━━━━━━━ 15/27 [aws-requests-auth]
    Found existing installation: opentelemetry-semantic-conventions 0.53b1/27 [aws-requests-auth]
    Uninstalling opentelemetry-semantic-conventions-0.53b1:━━━ 15/27 [aws-requests-auth]

### Environment Variables Setup

**Important:** Before running this workshop, you need to set up your AWS credentials and region. 


## Step 2: Configuration Files

Our application uses configuration files to manage settings. Let's create and understand these configurations.

In [ ]:
# Let's create our configuration files

# AWS Configuration
aws_config_content = '''
"""AWS Bedrock configuration settings."""
import os

AWS_CONFIG = {
    "region": "us-west-2",
    "model_id": "us.mistral.pixtral-large-2502-v1:0",  # Mistral model on Bedrock
}
'''


# MCP Server Configuration  
server_config_content = '''
"""MCP Server configurations for different tools."""
from mcp import StdioServerParameters
import boto3

# Get credentials from the execution role
session = boto3.Session()
credentials = session.get_credentials()

# Configuration for different MCP servers
SERVER_CONFIGS = [
    # Time utilities server
    StdioServerParameters(
        command="npx",
        args=["-y", "time-mcp"]
    ),
    # AWS documents 
    StdioServerParameters(
        command="uvx",
        args=["awslabs.aws-documentation-mcp-server@latest"],
        env= {
        "FASTMCP_LOG_LEVEL": "ERROR",
        "AWS_DOCUMENTATION_PARTITION": "aws"
      },
    ),
    # Uncomment and configure if you have Google Maps API key
    StdioServerParameters(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-google-maps"],
        env={"GOOGLE_MAPS_API_KEY": "<Google_API_KEY>"}
    ),
]
'''

# Write configuration files
with open('config.py', 'w') as f:
    f.write(aws_config_content)
    
with open('server_configs.py', 'w') as f:
    f.write(server_config_content)
    
print("✅ Configuration files created!")
print("📁 config.py - AWS Bedrock settings")
print("📁 server_configs.py - MCP server configurations")

✅ Configuration files created!
📁 config.py - AWS Bedrock settings
📁 server_configs.py - MCP server configurations


## Step 3: Understanding Model Context Protocol (MCP) and building MCP components 

**What is MCP?**
The Model Context Protocol is a standard protocol that allows AI models to  connect to external tools and data sources. 


**How it works:**
1. **MCP Host**:Maintains conversation, initiates MCP clients and has helper functions such as supporting image processing.
2. **MCP Clients**: Communication component within the host that connects to MCP servers.
3. **MCP Servers**: Connect to specific data sources (files, APIs, databases) and serve that data back through the protocol, providing specific tools (like time, maps,file access).
4. **Bedrock AI Models**: Provide LLM capabilities to understand user questions, decide which MCP tools to use, and generate the final answer.

<img src="architecture.png" width="800px" alt="Architecture diagram">


Let's see what MCP servers and tools are available:

In [3]:

from config import AWS_CONFIG
from server_configs import SERVER_CONFIGS

print("🤖 AWS Configuration:")
for key, value in AWS_CONFIG.items():
    print(f"   {key}: {value}")

print(f"\n🔧 MCP Servers Configured: {len(SERVER_CONFIGS)}")
for i, server in enumerate(SERVER_CONFIGS, 1):
    print(f"   Server {i}: {' '.join(server.args)}")

🤖 AWS Configuration:
   region: us-west-2
   model_id: us.mistral.pixtral-large-2502-v1:0

🔧 MCP Servers Configured: 3
   Server 1: -y time-mcp
   Server 2: awslabs.aws-documentation-mcp-server@latest
   Server 3: -y @modelcontextprotocol/server-google-maps


## Next let's set up the Bedrock Model, and MCP clients. 

1. **BedrockModel** - Connects to Amazon Bedrock for AI language model access (e.g. Mistral Models)
2. **MCPClient** -Communicates with MCP servers to access external data sources, APIs, and tools.


In [4]:
# Import all necessary libraries
import asyncio
from strands import Agent
from strands.tools.mcp import MCPClient
from strands.models import BedrockModel
from mcp import stdio_client
from datetime import datetime
from contextlib import ExitStack


In [5]:
# Create the Bedrock model

model_id = AWS_CONFIG["model_id"]
print(f"🚀 Initializing Bedrock model: {model_id}")

bedrock_model = BedrockModel(
    model_id=model_id,
    streaming=False  # We'll use non-streaming for Mistral models
)

print("✅ Bedrock model initialized successfully!")
    

🚀 Initializing Bedrock model: us.mistral.pixtral-large-2502-v1:0
✅ Bedrock model initialized successfully!


Now let's connect to our MCP tool servers and see what tools they provide:

In [6]:
# Set up MCP clients and tools
print("🔧 Setting up MCP clients...")

# Create MCP clients for each server configuration
mcp_clients = [
    MCPClient(lambda cfg=server_config: stdio_client(cfg))
    for server_config in SERVER_CONFIGS
]


# Use ExitStack to manage the lifecycle of our MCP connections
exit_stack = ExitStack()

try:
    # Connect to all MCP servers
    for i, mcp_client in enumerate(mcp_clients):
        exit_stack.enter_context(mcp_client)
        print(f"✅ Connected to MCP server {i+1}")
    
    # Collect all available tools
    tools = []
    for i, mcp_client in enumerate(mcp_clients):
        try:
            client_tools = mcp_client.list_tools_sync()
            tools.extend(client_tools)
            print(f"🔨 Loaded {len(client_tools)} tools from server {i+1}")
        except Exception as e:
            print(f"⚠️ Error getting tools from server {i+1}: {e}")
    
    print(f"\n🎉 Total tools available: {len(tools)}")
    
    # Display available tools
    if tools:
        print("\n🔧 Available Tools:")
        for tool in tools:
            tool_spec = tool.tool_spec
            # print(f"   • {tool_spec['name']}: {tool_spec['description']}")
            print(f"   • {tool_spec['name']}")
    
except Exception as e:
    print(f"❌ Error setting up MCP tools: {e}")
    tools = []

🔧 Setting up MCP clients...
✅ Connected to MCP server 1
✅ Connected to MCP server 2
✅ Connected to MCP server 3
🔨 Loaded 6 tools from server 1
🔨 Loaded 3 tools from server 2
🔨 Loaded 7 tools from server 3

🎉 Total tools available: 16

🔧 Available Tools:
   • current_time
   • relative_time
   • days_in_month
   • get_timestamp
   • convert_time
   • get_week_year
   • read_documentation
   • search_documentation
   • recommend
   • maps_geocode
   • maps_reverse_geocode
   • maps_search_places
   • maps_place_details
   • maps_distance_matrix
   • maps_elevation
   • maps_directions


### Creating the Strands Agent and chat loop 

Now we combine our Bedrock model with the MCP tools to create an intelligent Strands agent:


In [7]:
# Create the AI agent
system_prompt = """
You are a helpful assistant that can use tools to help you answer questions and perform tasks.
Be friendly, helpful, and make use of your tools when appropriate.
"""

agent = Agent(
    model=bedrock_model, 
    tools=tools,
    system_prompt=system_prompt
)
print("🤖 AI Agent created successfully!")
print(f"🧠 Agent has access to {len(tools)} tools")

🤖 AI Agent created successfully!
🧠 Agent has access to 16 tools


Let's test our agent with a simple chat loop to make sure everything is working:

In [8]:
# print("💬 Chat with the assistant. Type 'quit' to exit.\n")

while True:
    user_input = input("👤 User: ")
    if user_input.strip().lower() == "quit":
        print("👋 Goodbye!")
        break

    # Replace this with your own agent/response logic
    response = agent(user_input)
    print()
    print()

    print(f"🤖 Agent: {response}")
    print("-" * 120)


👤 User:  what's the time in New York now



Tool #1: current_time
The time in New York is currently 10:43 AM.

🤖 Agent: The time in New York is currently 10:43 AM.

------------------------------------------------------------------------------------------------------------------------


👤 User:  suggest top3 stores in london



Tool #2: maps_search_places
Here are the top 3 stores in London:

1. **Liberty London**
   - Rating: 4.5
   - Address: Regent St., Carnaby, London W1B 5AH, United Kingdom
   - Types: Department store, Tourist attraction, Hair care, Clothing store, Beauty salon, Spa, Store, Restaurant, Food, Point of interest, Health, Establishment

2. **Selfridges**
   - Rating: 4.5
   - Address: 400 Oxford St, London W1A 1AB, United Kingdom
   - Types: Department store, Cafe, Meal takeaway, Grocery or supermarket, Home goods store, Bar, Store, Restaurant, Food, Point of interest, Establishment

3. **Harrods**
   - Rating: 4.4
   - Address: 87-135 Brompton Rd, London SW1X 7XL, United Kingdom
   - Types: Department store, Tourist attraction, Jewelry store, Clothing store, Store, Point of interest, Establishment

🤖 Agent: Here are the top 3 stores in London:

1. **Liberty London**
   - Rating: 4.5
   - Address: Regent St., Carnaby, London W1B 5AH, United Kingdom
   - Types: Department store, Tourist att

👤 User:  is harrod open now?



Tool #3: maps_place_details
Yes, Harrods is open now. Here are the current opening hours:

- Monday: 10:00 AM – 9:00 PM
- Tuesday: 10:00 AM – 9:00 PM
- Wednesday: 10:00 AM – 9:00 PM
- Thursday: 10:00 AM – 9:00 PM
- Friday: 10:00 AM – 9:00 PM
- Saturday: 10:00 AM – 9:00 PM
- Sunday: 11:30 AM – 6:00 PM

🤖 Agent: Yes, Harrods is open now. Here are the current opening hours:

- Monday: 10:00 AM – 9:00 PM
- Tuesday: 10:00 AM – 9:00 PM
- Wednesday: 10:00 AM – 9:00 PM
- Thursday: 10:00 AM – 9:00 PM
- Friday: 10:00 AM – 9:00 PM
- Saturday: 10:00 AM – 9:00 PM
- Sunday: 11:30 AM – 6:00 PM

------------------------------------------------------------------------------------------------------------------------


👤 User:  can you give me supported mistral model ID on bedrock 



Tool #4: search_documentation

Tool #5: read_documentation

Tool #6: read_documentation

Tool #7: read_documentation

Tool #8: read_documentation

Tool #9: read_documentation

Tool #10: read_documentation
The supported Mistral model IDs on Amazon Bedrock are:

1. **Mistral 7B Instruct**
   - Model ID: `mistral.mistral-7b-instruct-v0:2`
   - Regions supported: `us-east-1, us-west-2, ap-south-1, ap-southeast-2, ca-central-1, eu-west-1, eu-west-2, eu-west-3, sa-east-1`
   - Input modalities: `Text`
   - Output modalities: `Text`
   - Streaming supported: `Yes`

2. **Mistral Large (24.02)**
   - Model ID: `mistral.mistral-large-2402-v1:0`
   - Regions supported: `us-east-1, us-west-2, ap-south-1, ap-southeast-2, ca-central-1, eu-west-1, eu-west-2, eu-west-3, sa-east-1`
   - Input modalities: `Text`
   - Output modalities: `Text`
   - Streaming supported: `Yes`

3. **Mistral Large (24.07)**
   - Model ID: `mistral.mistral-large-2407-v1:0`
   - Regions supported: `us-west-2`
   - Input moda

👤 User:  quit


👋 Goodbye!


### test questions: 
- can you give me mistral models model ID on bedrock 

## Step 4: Building a Web Interface with Gradio

Now let's take our chat application to the next level by creating a beautiful web interface using Gradio. This will allow users to:

- 💬 Chat with the AI in a web browser
- 🖼️ Upload and analyze images
- 🔧 View available tools and configuration
- 📱 Access from any device

### Understanding Gradio

Gradio is a Python library that makes it easy to create web interfaces for machine learning models and applications. With just a few lines of code, we can create:
- Chat interfaces
- File upload areas
- Information panels

In [9]:
# Import Gradio and other necessary libraries for the web interface
import gradio as gr
from PIL import Image
import base64
import io
import json


### Helper Functions for the Web Interface

Let's create some helper functions for our web interface:

In [10]:
def get_available_tools():
    """Get list of available tools for display"""
    global tools
    if not tools:
        return "No tools available yet."
    
    html = "<h3>Available Tools</h3>"
    for tool in tools:
        try:
            tool_spec = tool.tool_spec
            name = tool_spec.get('name', 'Unknown Tool')
            description = tool_spec.get('description', 'No description available')
            
            html += f"""
            <div style="border: 1px solid #ddd; margin-bottom: 10px; padding: 10px; border-radius: 5px;">
                <div style="font-weight: bold; color: #2C3E50;">{name}</div>
                <div style="margin-top: 5px;">{description}</div>
            </div>
            """
        except Exception as e:
            logger.error(f"Error displaying tool: {str(e)}")
            html += f"""
            <div style="border: 1px solid #ddd; margin-bottom: 10px; padding: 10px; border-radius: 5px; color: red;">
                <div>Error displaying tool: {str(e)}</div>
            </div>
            """
    
    return html


def convert_image_to_bytes(image):
    """Convert PIL Image to bytes for Bedrock message format"""
    if image is None:
        return None
    
    try:
        # Convert PIL Image to bytes
        buffered = io.BytesIO()
        # Determine format based on image
        format_type = image.format if image.format else 'PNG'
        if format_type not in ['PNG', 'JPEG', 'JPG']:
            format_type = 'PNG'
        
        image.save(buffered, format=format_type)
        image_bytes = buffered.getvalue()
        
        return {
            'bytes': image_bytes,
            'format': format_type.lower()
        }
    except Exception as e:
        logger.error(f"Error converting image to bytes: {e}")
        return None


def process_message(message, image=None):
    """Process a message from the user and get a response from the agent"""
    global agent, tools
    
    if agent is None:
        # First-time initialization
        if not initialize_agent():
            return "Error: Failed to initialize the agent. Please check the logs."
    
    try:
        # Handle image input by appending message to agent
        if image is not None:
            # Convert image to bytes
            image_data = convert_image_to_bytes(image)
            if image_data:
                # Create message with image and text content
                new_message = {
                    "role": "user",
                    "content": [
                        {
                            "image": {
                                "format": image_data['format'],
                                "source": {
                                    "bytes": image_data['bytes']
                                }
                            }
                        },
                        {
                            "text": message if message.strip() else "Please analyze the content of the image."
                        }
                    ]
                }
                
                # Append the new message to the agent's messages
                agent.messages.append(new_message)
                
                # Get response from agent
                response = agent(message)
            else:
                # Fallback to text-only if image conversion failed
                response = agent(message)
        else:
            # Text-only message
            response = agent(message)
        
        
        # Extract the text content from the agent result
        if hasattr(response, 'text'):
            display_response = response.text
        elif hasattr(response, 'content'):
            display_response = response.content
        else:
            # Fallback to string representation
            display_response = str(response)
        
        return display_response
        
    except Exception as e:
        logger.error(f"Error processing message: {e}")
        import traceback
        traceback.print_exc()
        return f"I encountered an error: {str(e)}"

def reset_conversation():
    """Reset conversation history"""
    global agent, tool_usage_history
    tool_usage_history = []
    return [], "Conversation has been reset."

def respond(message, chat_history, image=None):
    """Process the message and update the chat history"""
    if not message.strip() and image is None:
        return chat_history, "", None
    
    # Create user message content
    user_content = message
    if image is not None:
        user_content += " [Image uploaded]"
    
    # Add user message to history
    chat_history.append({"role": "user", "content": user_content})
    
    # Process user message with image
    bot_response = process_message(message, image)
    
    # Add assistant response to history
    chat_history.append({"role": "assistant", "content": bot_response})
    
    return chat_history, "", None  # Return empty message and clear image

### Building the Gradio Interface

Now let's create our beautiful web interface:

In [11]:
# Create the Gradio interface
with gr.Blocks(css="""
    .container {
        max-width: 1200px;
        margin: auto;
    }
    .tools-panel {
        background-color: #f9f9f9;
        border-radius: 10px;
        padding: 15px;
        margin-top: 15px;
    }
""", title="MCP Application Demo with Mistral Models on Amazon Bedrock") as demo:
    
    gr.HTML("""
        <div style="text-align: center; margin-bottom: 1rem">
            <h1>🤖 MCP Application Demo with Mistral Models on Amazon Bedrock</h1>
            <p>Chat with AI powered by Mistral models on Amazon Bedrock and MCP tools</p>
        </div>
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            # Chat interface
            chatbot = gr.Chatbot(
                height=500,
                show_label=False,
                container=True,
                show_copy_button=True,
                type="messages"
            )
            
            with gr.Row():
                with gr.Column(scale=6):
                    msg = gr.Textbox(
                        placeholder="Type your message here...",
                        show_label=False,
                        container=False
                    )
                with gr.Column(scale=3):
                    img_input = gr.Image(
                        type="pil",
                        label="Upload Image",
                        sources=["upload", "clipboard"],
                    )
                submit = gr.Button("Send", scale=1, variant="primary")
            
            with gr.Row():
                clear = gr.Button("Clear Chat", variant="secondary")
        
        with gr.Column(scale=1):
            # Tools panel with tabs
            with gr.Tab("Available Tools"):
                tools_display = gr.HTML(
                    value=get_available_tools,
                    show_label=False
                )
                refresh_tools = gr.Button("Refresh Tools", size="sm")
            
            with gr.Tab("Configuration"):
                gr.Markdown("### Current Configuration")
                with gr.Group():
                    gr.Textbox(
                        label="AWS Region",
                        value=AWS_CONFIG["region"],
                        interactive=False
                    )
                    gr.Textbox(
                        label="Model ID", 
                        value=AWS_CONFIG["model_id"],
                        interactive=False
                    )
                    gr.Markdown("*To change configuration, edit config.py and restart*")
    
    # Event handlers
    msg.submit(
        fn=respond,
        inputs=[msg, chatbot, img_input],
        outputs=[chatbot, msg, img_input]
    ).then(
        fn=get_available_tools,
        inputs=None,
        outputs=tools_display
    )
    
    submit.click(
        fn=respond,
        inputs=[msg, chatbot, img_input], 
        outputs=[chatbot, msg, img_input]
    ).then(
        fn=get_available_tools,
        inputs=None,
        outputs=tools_display
    )
    
    clear.click(
        fn=reset_conversation,
        inputs=None,
        outputs=[chatbot, gr.Textbox()]
    )
    
    refresh_tools.click(
        fn=get_available_tools,
        inputs=None,
        outputs=tools_display
    )

### Launch Your Web Application!

Now for the exciting part - let's launch your web application!

In [12]:
demo.queue()
demo.launch(
    share=True,
    server_name="0.0.0.0",
    server_port=7861,
    show_error=True
)

* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://1e1ead50134c0972d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



Tool #11: current_time
The time in London is currently 3:47 PM.

## 🎉 Congratulations! Workshop Complete!

You've successfully built a complete AI chat application with:

### ✅ What You've Accomplished:
1. **Set up** Amazon Bedrock integration
2. **Connected** MCP tool servers
3. **Created** an intelligent AI agent
4. **Built** a command-line chat interface
5. **Deployed** a beautiful web application
6. **Added** image analysis capabilities